In [1]:
!pip install torch_geometric torch gdown  --quiet

In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 81 (delta 7), reused 0 (delta 0), pack-reused 62 (from 2)
Receiving objects: 100% (81/81), 105.83 MiB | 45.52 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [2]:
%cd hackaton/

/home/onyxia/work/DL-Hackathon/hackaton


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets

Retrieving folder contents
Retrieving folder 1wcUVBNQkZ04zStXkglXSgERfIvjSHJiL A
Processing file 1C8sjkO6JS0j2SyVwQ07m8PhQ-pHpuI78 test.json.gz
Processing file 12N11n8gufNA_C1ns-1IeBseBHgrSfRI1 train.json.gz
Retrieving folder 1Tj5YoYYDDXjDxxi-cywZgoDkT0b1Qbz- B
Processing file 11GBlrXMdP3HSD60w-56Tu6rbGkR-Ifww test.json.gz
Processing file 13vp-Kwef3UgAwMG-dokGwKyARym9iqtL train.json.gz
Retrieving folder 1e3B_tBMd693Iwv8x3zRR9c47l5yt_5ey C
Processing file 18XVe65ZsQ0PDLCqQa4WmneVhyfjGcXmT test.json.gz
Processing file 1z5lvG2CytbLQZt7Jmo9BopzFd0pKejEj train.json.gz
Retrieving folder 1cvM0eZwpD4gzjo44_zdodxudVBMrLza1 D
Processing file 1Gna_dHnBLX8vKaYGAAqAbw5QPerrNK1u test.json.gz
Processing file 1Pc-6LMML80-AgEoLVs2Q5hLtmR_rTEek train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1C8sjkO6JS0j2SyVwQ07m8PhQ-pHpuI78
From (redirected): https://drive.google.com/uc?

In [3]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 24 07:10 A
drwxr-sr-x 2 onyxia users 4.0K May 24 07:11 B
drwxr-sr-x 2 onyxia users 4.0K May 24 07:11 C
drwxr-sr-x 2 onyxia users 4.0K May 24 07:11 D


In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed(42)

In [5]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [6]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [7]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [8]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))

    os.makedirs(submission_folder, exist_ok=True)

    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")

    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })

    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [9]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [10]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")

        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue

        if user_input == "" and default is not None:
            return default

        if user_input == "" and not required:
            return None

        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [11]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE), 3 (Generalized CE)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)
    args['q_GCE'] = get_user_input("q (used if baseline_mode=3)", default=0.7, type_cast=float)


    return argparse.Namespace(**args)


In [24]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/B/train.json.gz
Path to the test dataset [None]:  datasets/B/test.json.gz
Number of checkpoints to save during training [None]:  10
Which GPU to use if any [1]:  1
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn
Dropout ratio [0.0]:  0.2
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  10
Baseline mode: 1 (CE), 2 (Noisy CE), 3 (Generalized CE) [1]:  1
Noise probability p (used if baseline_mode=2) [0.2]:  
q (used if baseline_mode=3) [0.7]:  


Arguments received:
train_path: datasets/B/train.json.gz
test_path: datasets/B/test.json.gz
num_checkpoints: 10
device: 1
gnn: gcn
drop_ratio: 0.2
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 10
baseline_mode: 1
noise_prob: 0.2
q_GCE: 0.7


In [13]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [14]:
class GeneralizedCrossEntropyLoss(torch.nn.Module):
    def __init__(self, q=0.7):  # q hyperparamètre entre 0 et 1
        super().__init__()
        self.q = q

    def forward(self, logits, targets):
        probs = torch.softmax(logits, dim=1)
        pt = probs.gather(1, targets.unsqueeze(1)).squeeze(1)  # prob des bonnes classes
        loss = (1 - pt ** self.q) / self.q
        return loss.mean()

In [25]:
script_dir = os.getcwd()
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3

if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
elif args.baseline_mode == 3:
    criterion = GeneralizedCrossEntropyLoss(args.q_GCE)
else:
    criterion = torch.nn.CrossEntropyLoss()


In [26]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [27]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [28]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size


    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)


        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

IOStream.flush timed out
Iterating training graphs: 100%|██████████| 140/140 [01:23<00:00,  1.68batch/s]


Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/B/model_B_epoch_1.pth


Iterating eval graphs: 100%|██████████| 35/35 [00:13<00:00,  2.57batch/s]
Epoch 1/10, Loss: 1.6878, Train Acc: 0.3656, Val Acc: 0.2054
Epoch 1/10, Loss: 1.6878, Train Acc: 0.3656, Val Acc: 0.2054
Epoch 1/10, Loss: 1.6878, Train Acc: 0.3656, Val Acc: 0.2054


Epoch 1/10, Loss: 1.6878, Train Acc: 0.3656, Val Acc: 0.2054
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_B_best.pth


Iterating training graphs: 100%|██████████| 140/140 [01:19<00:00,  1.76batch/s]


...

Epoch 9/10, Loss: 1.4207, Train Acc: 0.5188, Val Acc: 0.4991
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_B_best.pth


Iterating training graphs: 100%|██████████| 140/140 [00:32<00:00,  4.27batch/s]


Checkpoint saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/B/model_B_epoch_10.pth


Iterating eval graphs: 100%|██████████| 35/35 [00:06<00:00,  5.45batch/s]
Epoch 10/10, Loss: 1.4009, Train Acc: 0.5297, Val Acc: 0.4795
Epoch 10/10, Loss: 1.4009, Train Acc: 0.5297, Val Acc: 0.4795
Epoch 10/10, Loss: 1.4009, Train Acc: 0.5297, Val Acc: 0.4795


Epoch 10/10, Loss: 1.4009, Train Acc: 0.5297, Val Acc: 0.4795


In [21]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

13426

In [22]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)


In [23]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

<ipython-input-23-10583d8b995c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))
Iterating eval graphs: 100%|██████████| 1

Predictions saved to /content/hackaton/submission/testset_A.csv


# Submission

In [24]:
import tarfile
import os

def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.

    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")

# Example usage
folder_path = "/content/hackaton/submission"            # Path to the folder you want to compress
output_file = "/content/hackaton/submission.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

Folder '/content/hackaton/submission' has been compressed into '/content/hackaton/submission.gz'


In [25]:
# Download the folder that we have to submit on HuggingFace
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>